In [200]:
import pandas as pd
from openpyxl import load_workbook

In [201]:
aggregated_df = pd.read_csv('../data/aggregated_dataset.csv')
aggregated_df.columns

Index(['Unnamed: 0', 'name', 'geometry', 'cost', 'beds', 'baths', 'parkings',
       'SA2_CODE21', 'SA2_NAME21', 'Nearest_station', 'Nearest_park',
       'Nearest_shop', 'Nearest_hospital', 'Nearest_school',
       'Nearest_supermarket'],
      dtype='object')

In [202]:
aggregated_df = aggregated_df.drop(columns=['Unnamed: 0'])

In [203]:
aggregated_df

,name,geometry,cost,beds,baths,parkings,SA2_CODE21,SA2_NAME21,Nearest_station,Nearest_park,Nearest_shop,Nearest_hospital,Nearest_school,Nearest_supermarket
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.9691204 -37.8095116),850.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,0.6452,1.7146,0.7007,2.1130,0.3461,0.743333
1,2302/17 Spring Street Melbourne VIC 3000,POINT (144.9744021 -37.8147493),1500.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,0.4319,2.7503,1.1439,2.4385,0.5171,2.190000
2,11/187 Collins Street Melbourne VIC 3000,POINT (144.9678522 -37.81540469999999),800.0,1.0,1.0,NaN,206041503,Melbourne CBD - East,2.1035,2.8712,1.2648,2.7744,0.2480,2.823333
3,1605/68 La Trobe Street Melbourne VIC 3000,POINT (144.9667899 -37.8083648),590.0,2.0,1.0,1.0,206041503,Melbourne CBD - East,1.0761,1.6096,0.9307,2.0254,0.4471,2.193333
4,3007/35 Spring Street Melbourne VIC 3000,POINT (144.9740049 -37.8141725),2300.0,3.0,2.0,2.0,206041503,Melbourne CBD - East,0.3607,2.6702,1.0638,2.2120,0.4370,1.968333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11669,21 Swallowtail Avenue Clyde North VIC 3978,POINT (145.3762985 -38.0840235),510.0,4.0,2.0,1.0,212031555,Clyde North - North,6.5979,6.0713,7.8176,28.0611,0.4323,10.038333
11670,18 Flowerbloom Crescent Clyde North VIC 3978,POINT (145.3448321 -38.0764965),650.0,4.0,2.0,2.0,212031555,Clyde North - North,5.1715,2.9896,4.7359,22.2783,1.4282,4.990000
11671,19 Anvil Way Clyde North VIC 3978,POINT (145.3604342 -38.08417439999999),475.0,3.0,2.0,2.0,212031555,Clyde North - North,5.7359,4.8191,6.5654,25.4616,1.2963,8.555000
11672,51 Springleaf Avenue Clyde North VIC 3978,POINT (145.3501506 -38.0813448),680.0,4.0,2.0,2.0,212031555,Clyde North - North,5.2711,3.2310,5.5270,24.9969,1.8010,6.766667


### Extract the Victoria history median renting price in 2011, 2016 and 2021

In [204]:
wb=load_workbook('../data/external_SA2/Housing costs.xlsx')
housing_2011 = wb['2.6 Rent_ILOC_2011']
housing_2016 = wb['2.10 Rent_ILOC_2016']
housing_2021 = wb['2.14 Rent_ILOC_2021']


In [205]:
# transform the sheets to dataframe
def read_sheet(sheet):
    data = sheet.values
    columns = next(data)
    return pd.DataFrame(data, columns=columns)

housing_history_2011 = read_sheet(housing_2011)
housing_history_2016 = read_sheet(housing_2016)
housing_history_2021 = read_sheet(housing_2021)


In [206]:
housing_history_2021.iloc[5:90]

,Australian Bureau of Statistics,None,None
5,New South Wales,None,None
6,Bogan,205,None
7,Dubbo - East,323,None
8,Dubbo - North,260,None
9,Dubbo - South,300,None
...,...,...,...
85,Clarence Valley,300,None
86,Grafton,300,None
87,Yamba,385,None
88,Yamba - Surrounds,315,None


In [207]:
# delete the header, rename columns
# 2011
housing_history_2011 = housing_history_2011.iloc[5:]
housing_history_2011.rename(columns={None: 'Weekly Median Rent','            Australian Bureau of Statistics': 'ABS'}, inplace=True)

# 2016
housing_history_2016 = housing_history_2016.iloc[5:]
housing_history_2016.rename(columns={None: 'Weekly Median Rent','            Australian Bureau of Statistics': 'ABS'}, inplace=True)
housing_history_2016 = housing_history_2016.iloc[:, :2] # as there are two columns name None, we need to delete the second

# 2021
housing_history_2021 = housing_history_2021.iloc[5:]
housing_history_2021.rename(columns={None: 'Weekly Median Rent', '            Australian Bureau of Statistics': 'ABS'}, inplace=True)
housing_history_2021 = housing_history_2021.iloc[:, :2]



In [208]:
def find_VIC_housing(df):
    df.reset_index(drop=True, inplace=True)
    row_index1 = df.loc[df["ABS"] == "Victoria"].index[0]
    row_index2 = df.loc[df["ABS"] == "Queensland"].index[0]
    sliced_df = df.iloc[row_index1: row_index2]
    return sliced_df

housing_history_2011 = find_VIC_housing(housing_history_2011)
housing_history_2016 = find_VIC_housing(housing_history_2016)
housing_history_2021 = find_VIC_housing(housing_history_2021)
    

In [209]:
housing_history_2011

,ABS,Weekly Median Rent
293,Victoria,None
294,Keilor,270
295,Sunshine,258
296,Cardinia,280
297,Craigieburn - Sunbury,281
...,...,...
378,Warrnambool,180
379,Wimmera,150
380,Alpine - Towong - Indigo,193
381,Wangaratta,170


In [210]:
housing_history_2016

,ABS,Weekly Median Rent
293,Victoria,None
294,Keilor,300
295,Sunshine,290
296,Cardinia,320
297,Craigieburn - Sunbury,322
...,...,...
378,Warrnambool,220
379,Wimmera,180
380,Alpine - Towong - Indigo,200
381,Wangaratta,200


In [211]:
housing_history_2021

,ABS,Weekly Median Rent
293,Victoria,None
294,Keilor,350
295,Sunshine,331
296,Cardinia,360
297,Craigieburn - Sunbury,380
...,...,...
378,Warrnambool,275
379,Wimmera,207
380,Alpine - Towong - Indigo,260
381,Wangaratta,250


### calculate the median renting price of each SA2 district

In [215]:
median_by_group = aggregated_df.groupby('SA2_NAME21')['cost'].median()

In [221]:
median_by_group

SA2_NAME21
Abbotsford      630.0
Airport West    575.0
Albert Park     642.5
Alexandra       325.0
Alfredton       440.0
                ...  
Yarram          400.0
Yarraville      625.0
Yarrawonga      500.0
Yarriambiack    357.5
Yea             400.0
Name: cost, Length: 494, dtype: float64

In [223]:
aggregated_list = list(aggregated_df['SA2_NAME21'])
history_list = list(housing_history_2011['ABS'])
common_values = set(aggregated_list).intersection(set(history_list))

### Our domain dataset includes 494 different SA2 districts, our historical median renting dataset includes 90 different SA2 districts

In [231]:
common_count = len(common_values)
group_count = len(median_by_group)
history_count = len(history_list)
print(common_count)
print(group_count)
print(history_count)


25
494
90
